# Get the data

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

# unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

## Inspect the data

In [ ]:
!ls pizza_steak/test/pizza

In [ ]:
import os

# Walk through pizza_steak directory
for dirpath, dirnames, filenames in os.walk("pizza_steak"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

In [ ]:
num_steak_images_train = len(os.listdir("pizza_steak/train/steak"))
num_steak_images_train

In [ ]:
# Get the classnames programmatically
import pathlib
import numpy as np

data_dir = pathlib.Path("pizza_steak/train")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names = class_names[1:]
class_names

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  target_folder = target_dir + target_class
  random_image = random.sample(os.listdir(target_folder), 1)

  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape: {img.shape}")

  return img

In [ ]:
# View random image
img = view_random_image(target_class="pizza", target_dir="pizza_steak/train/")

In [ ]:
tf.constant(img)

In [ ]:
img.shape # returns width, height, color channels

In [ ]:
# Get all the pixel values between 0 & 1
img/255

## An End to End Example

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# set seed
tf.random.set_seed(42)

# Preprocess data
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup paths to our data directories
train_dir = "/content/pizza_steak/train"
test_dir = "pizza_steak/test"

# Import data from dirs and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode="binary",
                                               seed=42)

# Build a CNN model
model_1 = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=10,
                         kernel_size=3,
                         activation="relu",
                         input_shape=(224, 224, 3)),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2,
                            padding="valid"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=(["accuracy"]))

history_1 = model_1.fit(train_data, epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

In [ ]:
model_1.summary()

In [ ]:
pd.DataFrame(history_1.history).plot()

## Using the same model as before

In [ ]:
# Get random seed
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid"),
])

model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_2 = model_2.fit(train_data,
            epochs=10,
            steps_per_epoch=len(train_data),
            validation_data=valid_data,
            validation_steps=len(valid_data))

In [ ]:
model_2.summary()

In [ ]:
pd.DataFrame(history_2.history).plot()

In [ ]:
tf.random.set_seed(42)

model_3 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(224, 224, 3)),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(100, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid")
])

model_3.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

history_3 = model_3.fit(
    train_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_data=valid_data,
    validation_steps=len(valid_data)
)

In [ ]:
model_3.summary()

In [ ]:
pd.DataFrame(history_3.history).plot()

In [ ]:
model_1.summary()

## Binary Classification: Break Down

In [ ]:
# Visualize the data
plt.figure()
plt.subplot(1, 2, 1)
steak_img = view_random_image("pizza_steak/train/", "steak")
plt.subplot(1, 2, 2)
pizza_img = view_random_image("pizza_steak/train/", "pizza")

## Preprocessing the data

In [ ]:
# Define directory dataset paths
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

In [ ]:
# Turn data into batches

# Craete train and test data generators and rescale the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
# Load in out image data from directories and turn them into batches

train_data = train_datagen.flow_from_directory(directory=train_dir, # Target directory if images
                                               target_size=(224, 224), # Target size of images (H, W)
                                               class_mode="binary", # Type of the data
                                               batch_size=32) # Size of the mini batches

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             target_size=(224, 224),
                                             class_mode="binary",
                                             batch_size=32)

In [ ]:
# Get a sample of a train data batch
images, labels = train_data.next() # get the nect batch of data
len(images), len(labels)

In [ ]:
# How many batches are there?
len(train_data)

In [ ]:
# Get the first two images
images[:2], images[0].shape

In [ ]:
index = random.randint(0, 31)
plt.imshow(images[index])
plt.axis("off")
plt.title("Pizza" if labels[index] == 0.0 else "Stack")

## Create a CNN model (Start with baseline)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

In [ ]:
model_4 = Sequential([
  Conv2D(filters=32,
         kernel_size=3,
         strides=1,
         padding="valid",
         activation="relu",
         input_shape=(224, 224, 3)),
  Conv2D(10, 3, activation="relu"),
  Conv2D(10, 3, activation="relu"),
  Flatten(),
  Dense(1, activation="sigmoid")
])

model_4.compile(loss="binary_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

history_4 = model_4.fit(train_data,
            epochs=10,
            steps_per_epoch=len(train_data),
            validation_data=test_data,
            validation_steps=len(test_data))

In [ ]:
pd.DataFrame(history_4.history).plot()

In [ ]:
model_4.summary()

## Evaluating a model

In [ ]:
model_4.evaluate(test_data)

In [ ]:
# Plot the validation and training curves separately
def plot_loss_curves(history):
  """
  Returns separate loss curves
  """
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.figure(figsize=(18,7))

  plt.subplot(1,2,1)
  plt.plot(epochs, loss, label="Training Loss")
  plt.plot(epochs, val_loss, label="Validation Loss")
  plt.title("loss")
  plt.xlabel("epochs")
  plt.legend()

  # Plot Accuracy
  plt.subplot(1,2,2)
  plt.plot(epochs, accuracy, label="Training Accuracy")
  plt.plot(epochs, val_accuracy, label="Validation Accuracy")
  plt.title("accuracy")
  plt.xlabel("epochs")
  plt.legend();

In [ ]:
plot_loss_curves(history_4)

## Adjust the model parameters

In [ ]:
model_5 = Sequential([
  Conv2D(32, 3, activation="relu", input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2),
  Conv2D(10, 3, activation="relu", input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2),
  Conv2D(19, 3, activation="relu", input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2),
  Flatten(),
  Dense(1, activation="sigmoid")
])

In [ ]:
model_5.compile(
    loss="binary_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

In [ ]:
history_5 = model_5.fit(
    train_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data)
)

In [ ]:
model_5.summary()

In [ ]:
plot_loss_curves(history_5)

## Adding Data Augmentation

In [ ]:
# Data with Augmentation
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=0.2,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True)
# Data without Augmentation
train_datagen = ImageDataGenerator(rescale=1/255.)

# Test data without Augmentation
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
print("Augmented Train Dataset")
train_data_augmented = train_datagen_augmented.flow_from_directory(train_dir,
                                                                   target_size=(224, 224),
                                                                   batch_size=32,
                                                                   class_mode="binary",
                                                                   seed=42)
print("Non-Augmented training data")
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode="binary",
                                               seed=42)

print("Non-augmented test data")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224, 224),
                                             batch_size=32,
                                             class_mode="binary")

In [ ]:
# Get sample data batch
images, labels = train_data.next()
augmented_images, augmented_labels = train_data_augmented.next()

In [ ]:
index = random.randint(0, 31)

plt.subplot(1,2,1)
plt.imshow(images[index])
plt.axis("off")
plt.title("Pizza" if labels[index] == 0.0 else "Steak")

plt.subplot(1,2,2)
plt.imshow(augmented_images[index])
plt.axis("off")
plt.title("Augmented Pizza" if labels[index] == 0.0 else "Augmented Steak");

## Training a model in Augmented data

In [ ]:
model_6 = Sequential([
  Conv2D(filters=32, kernel_size=3, strides=1, padding="valid", input_shape=(224, 224, 3), activation="relu"),
  MaxPool2D(pool_size=2),
  Conv2D(filters=32, kernel_size=3, strides=1, padding="valid", input_shape=(224, 224, 3), activation="relu"),
  MaxPool2D(pool_size=2),
  Conv2D(filters=32, kernel_size=3, strides=1, padding="valid", input_shape=(224, 224, 3), activation="relu"),
  MaxPool2D(pool_size=2),
  Flatten(),
  Dense(1, activation="sigmoid")
])

In [ ]:
model_6.compile(
    loss="binary_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

In [ ]:
history_6 = model_6.fit(
    train_data_augmented,
    epochs=10,
    steps_per_epoch=len(train_data_augmented),
    validation_data=test_data,
    validation_steps=len(test_data)
)

In [ ]:
model_6.summary()

In [ ]:
plot_loss_curves(history_6)

In [ ]:
plot_loss_curves(history_5)

## Making a prediction with our trained model on our own custom data

In [ ]:
print(class_names)

In [ ]:
# View our example images

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-steak.jpeg

steak =  mpimg.imread("03-steak.jpeg")
plt.imshow(steak)
plt.axis(False)

In [ ]:
steak.shape

In [ ]:
# Create a function to import an image and resize it to be able to use with our image

def load_and_prep_image(filename, img_shape=224):
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode the read file in to a tensor
  img = tf.image.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, size=[img_shape, img_shape])
  # Rescale the image
  img = img/255.
  return img

In [ ]:
steak = load_and_prep_image("03-steak.jpeg")

In [ ]:
steak

In [ ]:
pred = model_6.predict(tf.expand_dims(steak, axis=0))
pred

In [ ]:
class_names

In [ ]:
pred_class = class_names[int(tf.round(pred))]
pred_class

In [ ]:
def pred_and_plot(model, filename, class_names=class_names):
  # Import the traget image and preprocess it
  img = load_and_prep_image(filename)

  # make prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the prediction classs
  pred_class = class_names[int(tf.round(pred))]
  
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False)

In [ ]:
pred_and_plot(model_6, "03-steak.jpeg")

In [ ]:
!wget -O steak_new.jpg https://st.depositphotos.com/1000504/4511/i/950/depositphotos_45119209-stock-photo-grilled-beef-steak.jpg

In [ ]:
pred_and_plot(model_6, "steak_new.jpg")

In [ ]:
!wget -O pizza_test.jpg https://www.oetker.in/Recipe/Recipes/oetker.in/in-en/pizza/image-thumb__52711__RecipeDetail/pizza-pollo-arrosto.jpg 

In [ ]:
pred_and_plot(model_6, "pizza_test.jpg")

## Multi-class Iamge Classification

### 1. Explore data

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

In [ ]:
# Unzip the data

zip_ref = zipfile.ZipFile("10_food_classes_all_data.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os

# Walk through data
for dirpath, dirnames, filenames in os.walk("10_food_classes_all_data"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

In [ ]:
!ls -la 10_food_classes_all_data/

In [ ]:
train_dir = "10_food_classes_all_data/train/"
test_dir = "10_food_classes_all_data/test/"

In [ ]:
import pathlib
import numpy as np

data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [ ]:
# visualization
import random
img = view_random_image(target_dir=train_dir,
                        target_class=random.choice(class_names))

### 2. Preprocess the data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# Load data from dirs and load it in batches
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

### 3. Create baseline model

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation

model_8 = Sequential([
  Conv2D(32, 3, input_shape=(224, 224, 3)),
  Activation(activation="relu"),
  Conv2D(32, 2, activation="relu"),
  MaxPool2D(),
  Conv2D(32, 3, activation="relu"),
  Conv2D(32, 3, activation="relu"),
  MaxPool2D(),
  Flatten(),
  Dense(10, activation="softmax")
])

model_8.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)


In [ ]:

history_8 = model_8.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=10,
    validation_data=test_data,
    validation_steps=len(test_data)
)

### 5. Evaluate the model

In [ ]:
plot_loss_curves(history_8)

In [ ]:
model_8.evaluate(test_data)

### 6. Adjust the model hyperparameters

In [ ]:
model_9  = Sequential([
  Conv2D(32, 3, input_shape=(224, 224, 3)),
  Activation(activation="relu")  ,
  MaxPool2D(),
  Conv2D(32, 3, activation="relu"),
  MaxPool2D(),
  Flatten(),
  Dense(10, activation="softmax")
])

model_9.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
history_9 = model_9.fit(
    train_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data)
)

In [ ]:
plot_loss_curves(history_9)

In [ ]:
model_9.summary()

In [ ]:
model_8.summary()

### Trying data augmentation

In [ ]:
# Create an augmented data generator instance
train_datagen_aug = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True     
)

train_data_aug = train_datagen_aug.flow_from_directory(train_dir,
                                      target_size=(224, 224),
                                      batch_size=32,
                                      class_mode="categorical")

In [ ]:
# Create model with augmented data

model_10 = tf.keras.models.clone_model(model_8)

model_10.compile(
    loss="categorical_crossentropy",
    optimizer=tf.optimizers.Adam(),
    metrics=["accuracy"]
)


In [ ]:
model_10.summary()

In [ ]:
history_10 = model_10.fit(
    train_data_aug,
    epochs=10,
    steps_per_epoch=len(train_data_aug),
    validation_data=test_data,
    validation_steps=len(test_data)
)

In [ ]:
plot_loss_curves(history_10)